<a href="https://colab.research.google.com/github/MittalNeha/Extensive_Vision_AI6/blob/main/week4/EVA6_Session_4_bhaskar_squeeze_n_expand2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [5]:
class Net(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 12, 3, padding=1)     #28 - 28 - 3x3
        self.conv2 = nn.Conv2d(12, 24, 3, padding=1)    #28 - 28 - 5x5
        self.pool1 = nn.MaxPool2d(2, 2)                 #28 - 14 - 10x10
        self.conv3 = nn.Conv2d(24, 48, 3, padding=1)   #14 - 14 - 12x12
        self.conv4 = nn.Conv2d(48, 96, 3, padding=1)  #14 - 14 - 14x14
        self.pool2 = nn.MaxPool2d(2, 2)                 #14 -  7 - 28x28
        self.conv5 = nn.Conv2d(96, 192, 3)             #7  -  5 - 30x30
        self.conv6 = nn.Conv2d(192, 384, 3)            #5 -   3 - 32x32
        self.conv7 = nn.Conv2d(384, 10, 3)             #3 -   1 - 34x34
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        x = F.relu(self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [6]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 28, 28]             120
            Conv2d-2           [-1, 24, 28, 28]           2,616
         MaxPool2d-3           [-1, 24, 14, 14]               0
            Conv2d-4           [-1, 48, 14, 14]          10,416
            Conv2d-5           [-1, 96, 14, 14]          41,568
         MaxPool2d-6             [-1, 96, 7, 7]               0
            Conv2d-7            [-1, 192, 5, 5]         166,080
            Conv2d-8            [-1, 384, 3, 3]         663,936
            Conv2d-9             [-1, 10, 1, 1]          34,570
Total params: 919,306
Trainable params: 919,306
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.57
Params size (MB): 3.51
Estimated Total Size (MB): 4.08
-------------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [7]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))          #In future check why this normalization works 
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [14]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)   #send data to device/GPU
        optimizer.zero_grad()                               #reset to zero gradient
        output = model(data)                                #apply model
        loss = F.nll_loss(output, target)                   #get loss
        loss.backward()                                     #propagate loss backward
        optimizer.step()                                    
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),   #adjusted the decimal place to print finer accuracy
        100. * correct / len(test_loader.dataset)))

In [26]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0777, Accuracy: 9739/10000 (97.39%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0438, Accuracy: 9860/10000 (98.60%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0299, Accuracy: 9907/10000 (99.07%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0312, Accuracy: 9915/10000 (99.15%)



loss=0.017463156953454018 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 39.62it/s]



Test set: Average loss: 0.0335, Accuracy: 9905/10000 (99.05%)



In [31]:
class Net2(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net2, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)     #28 - 28 - 3x3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)    #28 - 28 - 5x5
        self.conv2_1 = nn.Conv2d(64, 32, 1)    #28 - 28 - 5x5
        self.pool1 = nn.MaxPool2d(2, 2)                 #28 - 14 - 10x10
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)   #14 - 14 - 12x12
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)  #14 - 14 - 14x14
        self.conv4_1 = nn.Conv2d(128, 32, 1)
        self.pool2 = nn.MaxPool2d(2, 2)                 #14 -  7 - 28x28
        self.conv5 = nn.Conv2d(32, 64, 3)             #7  -  5 - 30x30
        self.conv6 = nn.Conv2d(64, 128, 3)            #5 -   3 - 32x32
        self.conv7 = nn.Conv2d(128, 10, 3)             #3 -   1 - 34x34
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1(F.relu(self.conv2_1(F.relu(self.conv2(F.relu(self.conv1(x)))))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = self.pool2(F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        x = F.relu(self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [32]:
model = Net2().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
            Conv2d-3           [-1, 32, 28, 28]           2,080
         MaxPool2d-4           [-1, 32, 14, 14]               0
            Conv2d-5           [-1, 64, 14, 14]          18,496
            Conv2d-6          [-1, 128, 14, 14]          73,856
            Conv2d-7           [-1, 32, 14, 14]           4,128
         MaxPool2d-8             [-1, 32, 7, 7]               0
            Conv2d-9             [-1, 64, 5, 5]          18,496
           Conv2d-10            [-1, 128, 3, 3]          73,856
           Conv2d-11             [-1, 10, 1, 1]          11,530
Total params: 221,258
Trainable params: 221,258
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [33]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1321, Accuracy: 9580/10000 (95.80%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0647, Accuracy: 9794/10000 (97.94%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0425, Accuracy: 9877/10000 (98.77%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0425, Accuracy: 9872/10000 (98.72%)



loss=0.001070643775165081 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 38.15it/s]



Test set: Average loss: 0.0370, Accuracy: 9889/10000 (98.89%)



In [11]:
class Net3(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net3, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 24, 3, padding=1)     #28 - 28 - 3x3
        self.conv2 = nn.Conv2d(24, 48, 3)               #28 - 26 - 5x5
        self.conv2_1 = nn.Conv2d(48, 64, 3)             #26 - 24 - 7x7
        self.conv2_2 = nn.Conv2d(64, 24, 1)             #24 - 24 - 7x7
        self.pool1 = nn.MaxPool2d(2, 2)                 #24 - 12 - 14x14
        self.conv3 = nn.Conv2d(24, 48, 3, padding=1)   #12 - 12 - 12x12
        self.conv4 = nn.Conv2d(48, 64, 3)              #12 - 10 - 14x14
        self.conv4_1 = nn.Conv2d(64, 24, 1)           #10 - 10 - 14x14
        self.pool2 = nn.MaxPool2d(2, 2)                 #10 -  5 - 28x28
        self.conv5 = nn.Conv2d(24, 48, 3)             #5  -  3 - 30x30
        self.conv6 = nn.Conv2d(48, 10, 3)            #3 -   1 - 32x32
        #self.conv7 = nn.Conv2d(128, 10, 3)             #3 -   1 - 34x34
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1(F.relu(self.conv2_2(F.relu(self.conv2_1(F.relu(self.conv2(F.relu(self.conv1(x)))))))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = self.pool2(F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        x = (self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        #x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [12]:
model = Net3().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 28, 28]             240
            Conv2d-2           [-1, 48, 26, 26]          10,416
            Conv2d-3           [-1, 64, 24, 24]          27,712
            Conv2d-4           [-1, 24, 24, 24]           1,560
         MaxPool2d-5           [-1, 24, 12, 12]               0
            Conv2d-6           [-1, 48, 12, 12]          10,416
            Conv2d-7           [-1, 64, 10, 10]          27,712
            Conv2d-8           [-1, 24, 10, 10]           1,560
         MaxPool2d-9             [-1, 24, 5, 5]               0
           Conv2d-10             [-1, 48, 3, 3]          10,416
           Conv2d-11             [-1, 10, 1, 1]           4,330
Total params: 94,362
Trainable params: 94,362
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [15]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1397, Accuracy: 9575/10000 (95.75%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0683, Accuracy: 9776/10000 (97.76%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0389, Accuracy: 9882/10000 (98.82%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0383, Accuracy: 9886/10000 (98.86%)



loss=0.0018614205764606595 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 37.63it/s]



Test set: Average loss: 0.0382, Accuracy: 9893/10000 (98.93%)



In [16]:
class Net4(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net4, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 24, 3, padding=1)     #28 - 28 - 3x3
        self.conv2 = nn.Conv2d(24, 36, 3)               #28 - 26 - 5x5
        self.conv2_1 = nn.Conv2d(36, 48, 3)             #26 - 24 - 7x7
        self.conv2_2 = nn.Conv2d(48, 24, 1)             #24 - 24 - 7x7
        self.pool1 = nn.MaxPool2d(2, 2)                 #24 - 12 - 14x14
        self.conv3 = nn.Conv2d(24, 36, 3, padding=1)               #12 - 10 - 12x12
        self.conv4 = nn.Conv2d(36, 48, 3)              #10 - 8 - 14x14
        self.conv4_1 = nn.Conv2d(48, 24, 1)           #8 - 8 - 14x14
        self.pool2 = nn.MaxPool2d(2, 2)                 #8 -  4 - 28x28
        self.conv5 = nn.Conv2d(24, 48, 3)             #4  -  2 - 30x30
        self.conv6 = nn.Conv2d(48, 10, 3)            #3 -   1 - 32x32
        #self.conv7 = nn.Conv2d(128, 10, 3)             #3 -   1 - 34x34
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1(F.relu(self.conv2_2(F.relu(self.conv2_1(F.relu(self.conv2(F.relu(self.conv1(x)))))))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = self.pool2(F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        x = (self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        #x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [17]:
model = Net4().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 28, 28]             240
            Conv2d-2           [-1, 36, 26, 26]           7,812
            Conv2d-3           [-1, 48, 24, 24]          15,600
            Conv2d-4           [-1, 24, 24, 24]           1,176
         MaxPool2d-5           [-1, 24, 12, 12]               0
            Conv2d-6           [-1, 36, 12, 12]           7,812
            Conv2d-7           [-1, 48, 10, 10]          15,600
            Conv2d-8           [-1, 24, 10, 10]           1,176
         MaxPool2d-9             [-1, 24, 5, 5]               0
           Conv2d-10             [-1, 48, 3, 3]          10,416
           Conv2d-11             [-1, 10, 1, 1]           4,330
Total params: 64,162
Trainable params: 64,162
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [18]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.3086, Accuracy: 1028/10000 (10.28%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1736, Accuracy: 9459/10000 (94.59%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0388, Accuracy: 9878/10000 (98.78%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0388, Accuracy: 9871/10000 (98.71%)



loss=0.007338423747569323 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 39.20it/s]



Test set: Average loss: 0.0326, Accuracy: 9893/10000 (98.93%)



In [19]:
class Net5(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net5, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 24, 3, padding=1)     #28 - 28 - 3x3
        self.conv2 = nn.Conv2d(24, 24, 3)               #28 - 26 - 5x5
        self.conv2_1 = nn.Conv2d(24, 36, 3)             #26 - 24 - 7x7
        self.conv2_2 = nn.Conv2d(36, 24, 1)             #24 - 24 - 7x7
        self.pool1 = nn.MaxPool2d(2, 2)                 #24 - 12 - 14x14
        self.conv3 = nn.Conv2d(24, 24, 3, padding=1)               #12 - 10 - 12x12
        self.conv4 = nn.Conv2d(24, 36, 3)              #10 - 8 - 14x14
        self.conv4_1 = nn.Conv2d(36, 24, 1)           #8 - 8 - 14x14
        self.pool2 = nn.MaxPool2d(2, 2)                 #8 -  4 - 28x28
        self.conv5 = nn.Conv2d(24, 36, 3)             #4  -  2 - 30x30
        self.conv6 = nn.Conv2d(36, 10, 3)            #3 -   1 - 32x32
        #self.conv7 = nn.Conv2d(128, 10, 3)             #3 -   1 - 34x34
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1(F.relu(self.conv2_2(F.relu(self.conv2_1(F.relu(self.conv2(F.relu(self.conv1(x)))))))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = self.pool2(F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        x = (self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        #x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [20]:
model = Net5().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 28, 28]             240
            Conv2d-2           [-1, 24, 26, 26]           5,208
            Conv2d-3           [-1, 36, 24, 24]           7,812
            Conv2d-4           [-1, 24, 24, 24]             888
         MaxPool2d-5           [-1, 24, 12, 12]               0
            Conv2d-6           [-1, 24, 12, 12]           5,208
            Conv2d-7           [-1, 36, 10, 10]           7,812
            Conv2d-8           [-1, 24, 10, 10]             888
         MaxPool2d-9             [-1, 24, 5, 5]               0
           Conv2d-10             [-1, 36, 3, 3]           7,812
           Conv2d-11             [-1, 10, 1, 1]           3,250
Total params: 39,118
Trainable params: 39,118
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [21]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.3097, Accuracy: 9039/10000 (90.39%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0787, Accuracy: 9750/10000 (97.50%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0347, Accuracy: 9884/10000 (98.84%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0347, Accuracy: 9893/10000 (98.93%)



loss=0.025540096685290337 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 36.92it/s]



Test set: Average loss: 0.0382, Accuracy: 9872/10000 (98.72%)



In [34]:
class Net6(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net6, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 24, 3)                #28 - 26 - 3x3
        self.conv2 = nn.Conv2d(24, 36, 3)               #26 - 24 - 5x5
        self.conv2_1 = nn.Conv2d(36, 48, 3)             #24 - 22 - 7x7
        self.conv2_2 = nn.Conv2d(48, 24, 1)             #22 - 22 - 7x7
        self.pool1 = nn.MaxPool2d(2, 2)                 #22 - 11 - 14x14
        self.conv3 = nn.Conv2d(24, 36, 3)              #11 - 9 - 12x12
        self.conv4 = nn.Conv2d(36, 48, 3)              #9 - 7 - 14x14
        self.conv4_1 = nn.Conv2d(48, 24, 1)           #7 - 7 - 14x14
        #self.pool2 = nn.MaxPool2d(2, 2)                 #8 -  4 - 28x28
        self.conv5 = nn.Conv2d(24, 48, 3)             #7  -  5 - 30x30
        #self.conv6 = nn.Conv2d(36, 48, 3)            #5 -   3 - 32x32
        #self.conv7 = nn.Conv2d(48, 10, 3)             #3 -   1 - 34x34
        self.gap = nn.AvgPool2d(kernel_size=5)
        self.fc1 = nn.Linear(48, 10)
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1(F.relu(self.conv2_2(F.relu(self.conv2_1(F.relu(self.conv2(F.relu(self.conv1(x)))))))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = (F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        #x = F.relu(self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        x = F.relu(self.conv5(x))
        print(x.shape)
        x = self.gap(x)
        print(x.shape)
        #x = x.view(x[0],-1)
        x = self.fc1(x)
        print(x.shape)
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        #x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [35]:
my_nn = Net6()
print(my_nn)

Net6(
  (conv1): Conv2d(1, 24, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(24, 36, kernel_size=(3, 3), stride=(1, 1))
  (conv2_1): Conv2d(36, 48, kernel_size=(3, 3), stride=(1, 1))
  (conv2_2): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(24, 36, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(36, 48, kernel_size=(3, 3), stride=(1, 1))
  (conv4_1): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1))
  (conv5): Conv2d(24, 48, kernel_size=(3, 3), stride=(1, 1))
  (gap): AvgPool2d(kernel_size=5, stride=5, padding=0)
  (fc1): Linear(in_features=48, out_features=10, bias=True)
)


In [36]:
model = Net6().to(device)
summary(model, input_size=(1, 28, 28))

torch.Size([2, 48, 5, 5])
torch.Size([2, 48, 1, 1])


RuntimeError: ignored

In [37]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)




  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



loss=2.300271511077881 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]


loss=2.3031694889068604 batch_id=1:   0%|          | 0/469 [00:00<?, ?it/s]


loss=2.3031694889068604 batch_id=1:   0%|          | 2/469 [00:00<00:27, 17.25it/s]


loss=2.3074357509613037 batch_id=2:   0%|          | 2/469 [00:00<00:27, 17.25it/s]


loss=2.300734043121338 batch_id=3:   0%|          | 2/469 [00:00<00:27, 17.25it/s] 


loss=2.302330493927002 batch_id=4:   0%|          | 2/469 [00:00<00:27, 17.25it/s]


loss=2.2971644401550293 batch_id=5:   0%|          | 2/469 [00:00<00:27, 17.25it/s]


loss=2.2971644401550293 batch_id=5:   1%|▏         | 6/469 [00:00<00:23, 19.87it/s]


loss=2.3035523891448975 batch_id=6:   1%|▏         | 6/469 [00:00<00:23, 19.87it/s]


loss=2.29

KeyboardInterrupt: ignored